# Getting __more__ stock data

In [1]:
import numpy as np
import pandas as pd
import dill
from sqlalchemy import create_engine, inspect
from tqdm import tqdm_notebook
from alpha_vantage.timeseries import TimeSeries

So I'm going to try another API provider, [Alpha Vantage](https://www.alphavantage.co/documentation/), to see if they offer a more complete listing of end of day stock prices. The number to beat is 110 companies.

...Lets get started by importing our set of tickers

In [2]:
tickSymbs = sorted(dill.load(open('setOfTickerSymbols.pkl', 'r')))

...and our Alpha Vantage API key, from an ignored file not pushed to my public repo

In [3]:
av_key_handle = open("alphavantage.apikey", "r")
ts = TimeSeries(key=av_key_handle.read().strip(), output_format='pandas')
av_key_handle.close()

...finally we can connect to the sqlite database and start pulling down data

In [4]:
engine = create_engine('sqlite:///capstone.db')

In [5]:
losers = []
winners = []
for ticker in tqdm_notebook(tickSymbs):
    try:
        df, meta = ts.get_daily(symbol=ticker, outputsize='full')
        df.to_sql(meta["2. Symbol"], engine, if_exists = 'replace')
    except:
        losers.append(ticker)
    else:
        winners.append(meta["2. Symbol"])
        '''print meta["2. Symbol"]
        print df.head()
        print df.tail()'''

In [6]:
print len(winners), winners
print len(losers), losers

277 ['AAAP', 'ABBV', 'ABIO', 'ACOR', 'ACRS', 'ACRX', 'ADHD', 'ADMP', 'ADMS', 'ADRO', 'ADXS', 'AERI', 'AEZS', 'AFMD', 'AGEN', 'AGIO', 'AGLE', 'AGN', 'AGRX', 'AGTC', 'AIMT', 'AKAO', 'AKBA', 'ALDR', 'ALDX', 'ALKS', 'ALNY', 'ALRN', 'ALXN', 'AMAG', 'AMGN', 'AMPH', 'AMRN', 'ANAB', 'ANIK', 'ANTH', 'APRI', 'AQXP', 'ARDM', 'ARDX', 'ARGS', 'ARGX', 'ARNA', 'ARQL', 'ARRY', 'AST', 'ATNM', 'ATNX', 'ATRS', 'AVEO', 'AVIR', 'AXON', 'AXSM', 'AZN', 'AZRX', 'BHVN', 'BIIB', 'BLPH', 'BLRX', 'BLUE', 'BMRN', 'BMY', 'BOLD', 'BPTH', 'BTX', 'BYSI', 'CALA', 'CAPR', 'CATB', 'CBIO', 'CDTX', 'CELG', 'CEMP', 'CERC', 'CFRX', 'CHMA', 'CHRS', 'CLBS', 'CLDX', 'CLRB', 'CLSD', 'CLSN', 'CNAT', 'CNCE', 'CORI', 'CORT', 'CPRX', 'CRBP', 'CRMD', 'CTIC', 'CYCC', 'CYTK', 'CYTR', 'DARE', 'DBVT', 'DCTH', 'DERM', 'DMTX', 'DRRX', 'DVAX', 'EARS', 'EBIO', 'EGLT', 'EGRX', 'EIGR', 'EPZM', 'ESPR', 'EVOK', 'EXEL', 'EYEG', 'FBIO', 'FCSC', 'FENC', 'FGEN', 'FLKS', 'FLXN', 'FOLD', 'FOMX', 'GALE', 'GALT', 'GBT', 'GEMP', 'GERN', 'GILD', 'GLMD', '

Lets check in the morning how she did, how far back can she read and what % of companies were successfully pulled?